In [1]:
from exceiver.models import Exceiver, ExceiverClassifier

In [2]:
model = Exceiver.load_from_checkpoint("pretrained_models/exceiver/pretrained_TS_exceiver.ckpt")
model_classifier = ExceiverClassifier.load_from_checkpoint("pretrained_models/exceiver_classifier/pretrained_TS_exceiver_classifier.ckpt")

Lightning automatically upgraded your loaded checkpoint from v1.6.5 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint pretrained_models/exceiver/pretrained_TS_exceiver.ckpt`
Lightning automatically upgraded your loaded checkpoint from v1.6.5 to v2.5.2. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint pretrained_models/exceiver_classifier/pretrained_TS_exceiver_classifier.ckpt`


In [17]:
from exceiver.datasets import ExceiverDataModule


In [54]:
from pathlib import  Path
from torch.utils.data import DataLoader, BatchSampler, SequentialSampler

data_path = Path("/scratch/saigum/exceiver/processed/guo")
dm = ExceiverDataModule(data_path=data_path, classify="subtype", batch_size=64, num_workers=0)
dm.prepare_data()
dm.setup(stage='fit')
dl = DataLoader(val_dataset, batch_size=None, batch_sampler=None, sampler=sampler, num_workers=0, pin_memory=True)
gpu = torch.
# Load model
_ = model.eval()
_ = model.to(gpu)


emb = []
for batch in tqdm(dl):
    with torch.no_grad():
        batch = [i.to(gpu) if i is not None else i for i in batch]
        gene_ids, gene_vals, mask_ids, mask_vals, key_padding_mask, classify = batch
        input_query = model.query_emb.repeat(len(gene_ids), 1, 1)
        latent, encoder_weights = model.encoder_attn_step(gene_ids, gene_vals, input_query, key_padding_mask)
        latent = model.process_self_attn(latent)
        emb.append(latent.reshape(batch_size, -1).cpu())

# Concatenate and save to disk
emb = torch.cat(emb).numpy()
joblib.dump(emb, f"/scratch/saigum/exceiver/emb.joblib")

# Covnvert to AnnData object
emb_adata = ad.AnnData(X=emb.copy(), obs=dm.val_adata.obs[:len(emb)], var=pd.DataFrame({"emb_dim": np.arange(emb.shape[1])}))

# Compute UMAP
sc.pp.pca(emb_adata)
print("Computed PCA.")
sc.pp.neighbors(emb_adata, use_rep="X_pca")
print("Computed neighborhood graph.")
sc.tl.umap(emb_adata)
print("Computed UMAP.")

# Save to disk
emb_adata.write_h5ad(f"/scratch/ukhan/ndcn/emb.h5ad")
print("Saved AnnData object to disk.")

n masked genes: 386
train_adata shape: (8637, 18626)
val_adata shape: (3702, 18626)
preparing class labels for subtype


(<exceiver.datasets.ExceiverDataset at 0x7f806afc92b0>,
 <exceiver.datasets.ExceiverDataset at 0x7f806cc96ad0>)

In [ ]:
import scanpy as sc 
import pandas as pd


Index(['A1BG', 'NAT2', 'ADA', 'CDH2', 'AKT3', 'GAGE12F', 'ZBTB11-AS1', 'MED6',
       'NR2E3', 'NAALAD2',
       ...
       'HNRNPDL', 'DMTF1', 'PPP4R1', 'CDH1', 'SLC12A6', 'PTBP3', 'KCNE2',
       'DGCR2', 'CASP8AP2', 'SCO2'],
      dtype='object', length=23459)


/scratch/saigum/test/lib/python3.13/site-packages/anndata/_core/anndata.py:1756: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
adata = sc.read_csv("datasets/Guo/GSE99254.tsv",delimiter="\t")
print(adata.obs_names)

In [11]:
from scipy import sparse as sp


In [28]:
adata.obs

""
NTH10-0616A
NTH11-0616A
NTH15-0616A
NTH17-0616A
NTH2-0616A
...
TTY63-20171219
TTY65-20171219
TTY7-20171219
TTY8-20171219


In [ ]:
subtype = subtype.reindex(adata.obs_names)
print(subtype)
print(len(set(subtype["sampleType"])))
print(len(set(subtype["majorCluster "])))

In [52]:
adata.write_h5ad("datasets/Guo/GSE99254.h5ad")

In [ ]:
if not sp.issparse(adata.X):
    adata.X  = sp.csr_matrix(adata.X)
adata.obs_names_make_unique()
adata.var_names_make_unique()
import pandas as pd
from sklearn.preprocessing import LabelEncoder
subtype = pd.read_csv("/scratch/saigum/exceiver/datasets/Guo/subtype.ann",delimiter="\t").set_index("UniqueCell_ID ")
subtype["cluster_no"] = LabelEncoder().fit_transform(subtype["sampleType"])
adata.obs["subtype"] = subtype.loc[adata.obs_names]["cluster_no"]
adata.write_h5ad(filename="datasets/Guo/GSE99254.h5ad")

In [16]:
fd = sc.read_h5ad("datasets/Guo/GSE99254.h5ad")
print(fd)
print(fd.X)

AnnData object with n_obs × n_vars = 12346 × 23459
<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 36284370 stored elements and shape (12346, 23459)>
  Coords	Values
  (0, 2)	223.0
  (0, 4)	57.0
  (0, 7)	1.0
  (0, 73)	1.0
  (0, 74)	64.0
  (0, 75)	1.0
  (0, 80)	6.0
  (0, 83)	168.0
  (0, 87)	90.0
  (0, 221)	10.0
  (0, 248)	1.0
  (0, 261)	308.0
  (0, 263)	1.0
  (0, 301)	23.0
  (0, 309)	56.0
  (0, 335)	4.0
  (0, 345)	2.0
  (0, 346)	92.0
  (0, 374)	45.0
  (0, 398)	280.0
  (0, 480)	32.0
  (0, 487)	31.0
  (0, 498)	168.0
  (0, 499)	349.0
  (0, 509)	453.0
  :	:
  (12345, 23289)	55.0
  (12345, 23295)	1.0
  (12345, 23310)	10.0
  (12345, 23315)	1.0
  (12345, 23331)	306.0
  (12345, 23337)	3.0
  (12345, 23341)	17.0
  (12345, 23342)	2.0
  (12345, 23349)	105.0
  (12345, 23353)	87.0
  (12345, 23357)	59.0
  (12345, 23371)	293.0
  (12345, 23382)	337.0
  (12345, 23389)	1.0
  (12345, 23390)	3.0
  (12345, 23391)	692.0
  (12345, 23405)	2.0
  (12345, 23406)	1.0
  (12345, 23424)	13.0
  (12345, 23